Trabalho para realizar a conexão na plataforma do Bling:
A versão 3 de sua API utiliza do mecanismo de autentificação Oauth2, infelizmente a empresa não tem suporte para desenvolver, devivo a isso decidio compartilhar esse trabalho em minha rede. Espero que esse código ajude a todos.

In [ ]:
#Bibliotecas:
import requests
from requests_oauthlib import OAuth2Session
import base64

In [ ]:
#Função que serve para recer os código de acesso e renovação de acesso.
def getToken(code, client_id, client_secret, redirect_response, refresh_token):
    #print('1')
    authorization_code = code
    token_url = 'https://www.bling.com.br/Api/v3/oauth/token'
    credentials = f'{client_id}:{client_secret}'
    base64_credentials = base64.b64encode(credentials.encode('utf-8')).decode('utf-8')
    if (refresh_token == None): #(Processo para pegar o primeiro código de acesso)
        #print('2')
        #Parâmetros para a solicitação POST
        data = {'grant_type': 'authorization_code', 'code': authorization_code,}
        #Cabeçalhos da solicitação POST
        headers = {'Content-Type': 'application/x-www-form-urlencoded', 'Authorization': f'Basic {base64_credentials}', 'Accept': 'application/json'}
        response = requests.post(token_url, data=data, headers=headers)
        # Manipulação de erros
        if response.status_code == 200:
            token_data = response.json()
            #print(response.status_code)
            #print('Token de acesso:', token_data)
        else:
            print(f"Erro na solicitação: {response.status_code}")
            print(response.text)
        #print('3')
        #token = oauth.fetch_token(token_url, authorization_response=redirect_response, client_secret=client_secret)
        #print('4')
        return token_data
    if (refresh_token != None): #(Processo de Renovação do código de acesso)
        #print('Refresh 1')
        data = {'grant_type': 'refresh_token','refresh_token': refresh_token,}
        #Cabeçalhos da solicitação POST
        headers = {'Content-Type': 'application/x-www-form-urlencoded', 'Authorization': f'Basic {base64_credentials}','Accept': 'application/json',}
        response = requests.post(token_url, data=data, headers=headers)
        if response.status_code == 200:
            token_data = response.json()
            #print(response.status_code)
            #print('Token de refresh:', token_data)
        else:
            print(f"Erro na solicitação: {response.status_code}")
            print(response.text)
        #token = oauth.fetch_token(token_url, authorization_response=redirect_response, client_secret=client_secret)
        return token_data

In [ ]:
#Primeiro Código a rodar:
#Substitua essas informações pelas suas credenciais e configurações específicas da API
client_id = 'COLOQUE O SEU CÓDIGO DE CLIENTE'
client_secret = 'COLOQUE O SEU CÓDIGO DE CLIENTE SECRETO'
redirect_uri = 'https://www.google.com.br/?hl=pt-BR' #Optei por usar a do google, não me perguntem o motivo, deixem cadastrado igual na página de API.

#URL de autorização fornecida pelo serviço OAuth (Link do Convite)
authorization_base_url = 'COLOQUE O SEU CÓDIGO DE CONVITE, SE UTILIZOU O GOOGLE TAMBÉM VAI SER DIRECIONADO PARA O GOOGLE.'

In [ ]:
#Segundo Código a Rodar:
#Crie uma sessão OAuth2 sem especificar o escopo
oauth = OAuth2Session(client_id, redirect_uri=redirect_uri)

#Gere a URL de autorização
authorization_url, state = oauth.authorization_url(authorization_base_url)

#Exiba a URL de autorização
print('Por favor, vá para', authorization_url)

#Solicite que o usuário insira a URL de redirecionamento
redirect_response = input('\n\n\nCole aqui a URL de redirecionamento:\n\n\n ')

#Separando o Code
aux = redirect_response
x = redirect_response.find('code=')
code = redirect_response[x:x+100]
y = code.find('&')
code = code[5:y]


#Solicita as credênciais de acesso.
refresh_token = None #Essa linha só vai se repetir agora.
token = getToken(code, client_id, client_secret, redirect_response, refresh_token)
access_token = token['access_token']
refresh_token = token['refresh_token']

In [ ]:
#Exemplo de utilização da API pedidos da plataforma.
api_url = 'https://www.bling.com.br/Api/v3/pedidos/vendas'

# Cabeçalhos da solicitação com o token de acesso
headers = {'Authorization': f'Bearer {access_token}', 'Content-Type': 'application/json',}

# Faça a solicitação GET com os cabeçalhos de autenticação
response = requests.get(api_url, headers=headers)

# Manipulação de erros e processamento da resposta
if response.status_code == 200:
    dados_api = response.json()
    #print('Dados da API:', dados_api)
else:
    print(f"Erro na solicitação: {response.status_code}")
    print(response.text)

#Gera um novo token (ISSO VAI PRECISAR FICAR EM TODAS AS APLICAÇÕES!!!)
token = getToken(code, client_id, client_secret, redirect_response, refresh_token)
access_token = token['access_token']
refresh_token = token['refresh_token']
dados_api